Pantaree Somcharoen (G00365378)

# Countdown Numbers Game Notebook
***

## What is a Countdown Game?
***

<b>Countdown</b> is a game show of letters, words & numbers. It is based on the long-running French game show Des chiffres et des lettres ("Numbers & Letters"). [[1]](https://british-game-show.fandom.com/wiki/Countdown)

<img src="https://149458054.v2.pressablecdn.com/wp-content/uploads/2017/12/countdown-game-printable-free.png" width=500 height=500 />

## Countdown Numbers game 
***



The Countdown Numbers problem is as follows: given a sequence of source numbers and the target number, the basic arithmetical operators (addition, subtraction,
multiplication and division) solves the problem.[[2]](https://www.cambridge.org/core/journals/journal-of-functional-programming/article/countdown-problem/FCF81E290B194D1ACBF699842D4D4CA5)

There are always six source numbers selected from the sequence $[1..10,1..10,25,50,75,100]$, the target number is randomly chosen from the range $100..999$ approximate solutions are acceptable, each number can only be used once, and
there is a time limit of 30 seconds.[[3]](https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.86.1168&rep=rep1&type=pdf)

In Countdown, contestants score 10 points if they achieve
a perfect solution, but if neither achieves this, then the contestant or
contestants with the closest answer scores 7 points if they are within
5 of the target, and 5 points if they are within 10.[[5]](http://doc.gold.ac.uk/aisb50/AISB50-S02/AISB50-S2-Colton-paper.pdf)


<img src="https://i.ytimg.com/vi/sCqNY-k9jho/maxresdefault.jpg" width=500 height=500 />

For example[[4]](https://arxiv.org/pdf/1502.05450.pdf), if the six numbers drawn are
1,1,4,5,6,7 and the number to find is 899 the answer is: <br>
<b>Operations Remaining</b><br>
$6 \times 5 = 30 {1,1,4,7,30}$ <br>
$30 + 1 = 31 {1,4,7,31}$   <br>
$4 \times 7 = 28 {1,28,31}$<br>
$28 + 1 = 29 {29,31}$<br>
$29 \times 31 = 899 {899}$<br>

## Discussion of the complexity of the Countdown Numbers Game
***

Because of the numerous possibilities and variables, the countdown numbers game is complicated. One of the factors is when generating the target number, there is no correct solution for that. Another one is the number of possibilities and ways to find solutions which result in the complexity of the game. Moreover, in the given time, 30 seconds, is impossible for most people to solve this puzzle.

While fairly difficult for most people, solving
instances of the puzzle is relatively easy for software, and there is an
abundance of online solvers available. However, Some of them are bugged or use
incomplete or incorrect algorithms. [[4]](https://arxiv.org/pdf/1502.05450.pdf)

For instance, this is the first published algorithm (Buisson [1980]) used a
simple decomposition mechanism.<br>


Give the following list <br> 
$\{3, 50, 7, 4, 75, 8\}$ with the target number is $822$

The algorithm would start from the solution
(822) and use a backward chaining approach in the Prolog way. However, not all operations were tried; at odd
steps, only addition and subtractions were used, while
at even steps only divisions were used. So the algorithm
would at the first step generate thirteen numbers: $822,
822 ± 3, 822 ± 50, . . . ,$ and then try to divide all of
them by the remaining $5$ (or $6$ if no number was added
or subtracted) numbers. If a division succeeds, the algorithm would then be applied recursively on the new
result with the remaining numbers. Here the solution
can be found by:

|   Operations  | Remaining | 
|-------------------------|---------|
| (822 + 50)/4=218 |  {3,7,75,8}  | 
| (218 + 7)/3=75  | {75,8}      |
| 75 - 75=0          | {8}       |

The complexity of this algorithm is very low. If we
have n numbers, we first generate $2n + 1$ numbers and
try to divide them by $n − 1$ numbers, so we have to do
$(2n + 1)(n − 1)$ trial divisions. At the next step, we
would have on the average $2(n − 2) + 1$ numbers to
divide by $n − 3$ numbers, so $(2(n − 2) + 1)(n − 3)$ trial
divisions, and so on

This algorithm, of course, has significant limitations. It is
impossible to compute solutions requiring intermediate
results, such as the first one presented in the Countdown Numbers Game section,
because $31$ and $29$ must be built independently before
multiplying them to have 899. It is even impossible to
find solutions with divisions. Moreover, this method
can only find the exact result. If it doesn’t exist, the
computation has to be restarted with the closest number
to the number to find as a new goal. [[4]](https://arxiv.org/pdf/1502.05450.pdf)

## Solving the Countdown Numbers Game
***

In [182]:
import operator
from itertools import combinations, permutations, product
from itertools import combinations_with_replacement
from more_itertools import distinct_permutations

In [173]:
OPERATORS = {
    "+": operator.add,
    "-": operator.sub,
    "*": operator.mul,
    "/": operator.floordiv,
}

In [174]:
def is_valid(skeleton):
    num_count = 0
    last_op_pos = -1
    for op_pos in skeleton:
        # All elements between operators must be numbers
        num_count += op_pos - last_op_pos - 1
        # Invalid if not enough numbers for operator
        if num_count < 2:
            return False
        num_count -= 1
        last_op_pos = op_pos
    return True

In [175]:
def evaluate(sequence, target):
    # Use stack to keep track of unused numbers
    stack = []
    solutions = []
    # Loop through sequence and evaluate result
    for i, s in enumerate(sequence):
        if isinstance(s, int):
            stack.append(s)
        else:
            if len(stack) < 2:
                break
            n1 = stack.pop(0)
            n2 = stack.pop(0)
            
            # If sequence results in invalid intermediate step, stop evaluation
            if s == "-" and n2 >= n1:
                break
            if s == '/' and n1 % n2 != 0:
                break
            
            r = OPERATORS[s](n1, n2)
            
            if r == target:
                solutions.append(sequence[:i+1])
            
            stack.append(r)
    return solutions

In [176]:
def distinct_ordered_combinations(iterable, r):
    seen = set()
    for combination in combinations_with_replacement(iterable, r):
        for permutation in permutations(combination):
            if permutation not in seen:
                yield permutation
                seen.add(permutation)

In [186]:
def solve(numbers, target):
    solutions = []
    n = len(numbers)
    sequence = [None for __ in range(2*n-1)]
    
    for skeleton in combinations(range(2*n-1), n-1):
        if is_valid(skeleton):
            for nums in distinct_permutations(numbers):
                for ops in distinct_ordered_combinations(OPERATORS, n-1):
                    # Build sequence from skeleton and number/operator choice
                    nums_iter = iter(nums)
                    ops_iter = iter(ops)
                    i = 0
                    for j in range(2*n-1):
                        if skeleton[i] == j:
                            sequence[j] = next(ops_iter)
                            i += 1
                        else:
                            sequence[j] = next(nums_iter)
                    new_solutions = evaluate(sequence, target)
                    solutions.extend(new_solutions)
    
    return solutions

In [187]:
NUMBERS = [100, 25, 9, 2, 10, 3]
TARGET = 235

In [188]:
solutions = solve(NUMBERS, TARGET)

In [190]:
#print(solutions)

#### Operators and Functions
***

In [127]:
# The + operator as a function.
operator.add(4, 5)

9

In [128]:
# The * operator as a function.
operator.mul(4, 5)

20

In [129]:
# The - operator as a function.
operator.sub(4, 5)

-1

In [130]:
# The / operator as a function.
operator.truediv(4, 5)

0.8

In [131]:
# The benefit of these is that they are first class objects.
# Note the +, -, *, / operators can't be put in lists.
ops = [operator.add, operator.mul, operator.sub, operator.truediv] * 5
ops

[<function _operator.add(a, b, /)>,
 <function _operator.mul(a, b, /)>,
 <function _operator.sub(a, b, /)>,
 <function _operator.truediv(a, b, /)>,
 <function _operator.add(a, b, /)>,
 <function _operator.mul(a, b, /)>,
 <function _operator.sub(a, b, /)>,
 <function _operator.truediv(a, b, /)>,
 <function _operator.add(a, b, /)>,
 <function _operator.mul(a, b, /)>,
 <function _operator.sub(a, b, /)>,
 <function _operator.truediv(a, b, /)>,
 <function _operator.add(a, b, /)>,
 <function _operator.mul(a, b, /)>,
 <function _operator.sub(a, b, /)>,
 <function _operator.truediv(a, b, /)>,
 <function _operator.add(a, b, /)>,
 <function _operator.mul(a, b, /)>,
 <function _operator.sub(a, b, /)>,
 <function _operator.truediv(a, b, /)>]

In [132]:
# Using permutations we can get all permutations 
# with replacement of five operations.

# We use a limit because their are a large number.
limit = 10


for q in it.permutations(ops * 5, 5):
  if limit == 0:
    break
  print(q)
  limit = limit - 1

(<built-in function add>, <built-in function mul>, <built-in function sub>, <built-in function truediv>, <built-in function add>)
(<built-in function add>, <built-in function mul>, <built-in function sub>, <built-in function truediv>, <built-in function mul>)
(<built-in function add>, <built-in function mul>, <built-in function sub>, <built-in function truediv>, <built-in function sub>)
(<built-in function add>, <built-in function mul>, <built-in function sub>, <built-in function truediv>, <built-in function truediv>)
(<built-in function add>, <built-in function mul>, <built-in function sub>, <built-in function truediv>, <built-in function add>)
(<built-in function add>, <built-in function mul>, <built-in function sub>, <built-in function truediv>, <built-in function mul>)
(<built-in function add>, <built-in function mul>, <built-in function sub>, <built-in function truediv>, <built-in function sub>)
(<built-in function add>, <built-in function mul>, <built-in function sub>, <built-in 

#### Permutations and Combinations [[6]](https://www.mathsisfun.com/combinatorics/combinations-permutations.html)
***

- When the order doesn't matter, it is a <b>Combination</b>.
    - <b>Repetition is Allowed</b>: such as coins in your pocket (5,5,5,10,10)
    - <b>No Repetition</b>: such as lottery numbers (2,14,15,27,30,33)
- When the order does matter it is a <b>Permutation</b>.
    - <b>Repetition is Allowed</b>: such as the lock above. It could be "333".
    - <b>No Repetition</b>: for example the first three people in a running race. You can't be first and second.

In [133]:
# Example of combinations.
# Order matters: no. Replacement: no.
L = [1, 2, 3, 4]
for c in it.combinations(L, 2):
  print(c)

(1, 2)
(1, 3)
(1, 4)
(2, 3)
(2, 4)
(3, 4)


In [134]:
# Example of combinations with replacement.
# Order matters: no. Replacement: yes.
L = [1, 2, 3, 4]
for c in it.combinations_with_replacement(L, 2):
  print(c)

(1, 1)
(1, 2)
(1, 3)
(1, 4)
(2, 2)
(2, 3)
(2, 4)
(3, 3)
(3, 4)
(4, 4)


In [135]:
# Example of permutations of size 2.
# Order matters: yes. Replacement: no.
L = [1, 2, 3, 4]
for c in it.permutations(L, 2):
  print(c)

(1, 2)
(1, 3)
(1, 4)
(2, 1)
(2, 3)
(2, 4)
(3, 1)
(3, 2)
(3, 4)
(4, 1)
(4, 2)
(4, 3)


In [136]:
# Example of products of length 2.
# Order matters: yes. Replacement: yes.
L = [1, 2, 3, 4]
for c in it.product(L, repeat=2):
  print(c)

(1, 1)
(1, 2)
(1, 3)
(1, 4)
(2, 1)
(2, 2)
(2, 3)
(2, 4)
(3, 1)
(3, 2)
(3, 3)
(3, 4)
(4, 1)
(4, 2)
(4, 3)
(4, 4)


In [137]:
# Using product to generate all lists of five operations.
ops = [operator.add, operator.mul, operator.sub, operator.truediv]
limit = 100
for q in it.product(ops, repeat=5):
  if limit == 0:
    break
  print(q)
  limit = limit - 1

(<built-in function add>, <built-in function add>, <built-in function add>, <built-in function add>, <built-in function add>)
(<built-in function add>, <built-in function add>, <built-in function add>, <built-in function add>, <built-in function mul>)
(<built-in function add>, <built-in function add>, <built-in function add>, <built-in function add>, <built-in function sub>)
(<built-in function add>, <built-in function add>, <built-in function add>, <built-in function add>, <built-in function truediv>)
(<built-in function add>, <built-in function add>, <built-in function add>, <built-in function mul>, <built-in function add>)
(<built-in function add>, <built-in function add>, <built-in function add>, <built-in function mul>, <built-in function mul>)
(<built-in function add>, <built-in function add>, <built-in function add>, <built-in function mul>, <built-in function sub>)
(<built-in function add>, <built-in function add>, <built-in function add>, <built-in function mul>, <built-in fun

#### Reverse Polish Notation
***

<b>Reverse Polish notation (RPN)</b> is a method for representing expressions in which the operator symbol is placed after the arguments being operated on.
[[7]](https://mathworld.wolfram.com/ReversePolishNotation.html)

In [138]:
# 3 4 5 * +
3 + 4 * 5

23

In [139]:
# 3 4 + 5 *
(3 + 4) * 5

35

In [140]:
# New random numbers game.
play_nos, target = new_numbers_game()
play_nos, target

([75, 50, 100, 2, 8, 5], 178)

In [141]:
# Orderings of pairs.
for pair in it.permutations(play_nos, 2):
  print(pair)

(75, 50)
(75, 100)
(75, 2)
(75, 8)
(75, 5)
(50, 75)
(50, 100)
(50, 2)
(50, 8)
(50, 5)
(100, 75)
(100, 50)
(100, 2)
(100, 8)
(100, 5)
(2, 75)
(2, 50)
(2, 100)
(2, 8)
(2, 5)
(8, 75)
(8, 50)
(8, 100)
(8, 2)
(8, 5)
(5, 75)
(5, 50)
(5, 100)
(5, 2)
(5, 8)


In [142]:
# Change the target to be something that will work for just two numbers.
target = max(play_nos) * min(play_nos)
target

200

In [143]:
%%timeit

# Operators.
ops = [operator.add, operator.sub, operator.mul, operator.truediv]

# All pair, op combs that hit target.
for nos, op in it.product(it.permutations(play_nos, 2), ops):
  if op(nos[0], nos[1]) == target:
    print(nos[0], str(op), nos[1])

100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-i

2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built

2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built

100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-i

2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built

...

100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-i

2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built

100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-i

2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built-in function mul> 2
2 <built-in function mul> 100
100 <built

100 <built-in function mul> 2
2 <built-in function mul> 100
713 µs ± 19.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [144]:
%%timeit

# Operators.
ops = [operator.add, operator.sub, operator.mul, operator.truediv]

def hits_target(z):
  nos, op = z
  return (op(nos[0], nos[1]) == target)

# All pair, op combs that hit target.
list(filter(hits_target, it.product(it.permutations(play_nos, 2), ops)))

28.2 µs ± 467 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [145]:
# All pair, op combs that hit target.
list(filter(lambda z: z[1](z[0][0], z[0][1]) == target, it.product(it.permutations(play_nos, 2), ops)))

[((100, 2), <function _operator.mul(a, b, /)>),
 ((2, 100), <function _operator.mul(a, b, /)>)]

In [146]:
# Operators.
ops = [operator.add, operator.sub, operator.mul, operator.truediv]

# Limit the output.
limit = 1100

# For the limit.
i = 0
# Orderings of pairs.
for play_nos, opers in it.product(it.permutations(play_nos), it.product(*([ops] * 5))):
  print(play_nos, opers)
  i = i + 1
  if i >= limit:
    break

(75, 50, 100, 2, 8, 5) (<built-in function add>, <built-in function add>, <built-in function add>, <built-in function add>, <built-in function add>)
(75, 50, 100, 2, 8, 5) (<built-in function add>, <built-in function add>, <built-in function add>, <built-in function add>, <built-in function sub>)
(75, 50, 100, 2, 8, 5) (<built-in function add>, <built-in function add>, <built-in function add>, <built-in function add>, <built-in function mul>)
(75, 50, 100, 2, 8, 5) (<built-in function add>, <built-in function add>, <built-in function add>, <built-in function add>, <built-in function truediv>)
(75, 50, 100, 2, 8, 5) (<built-in function add>, <built-in function add>, <built-in function add>, <built-in function sub>, <built-in function add>)
(75, 50, 100, 2, 8, 5) (<built-in function add>, <built-in function add>, <built-in function add>, <built-in function sub>, <built-in function sub>)
(75, 50, 100, 2, 8, 5) (<built-in function add>, <built-in function add>, <built-in function add>, <bu

(75, 50, 100, 2, 8, 5) (<built-in function truediv>, <built-in function mul>, <built-in function truediv>, <built-in function sub>, <built-in function sub>)
(75, 50, 100, 2, 8, 5) (<built-in function truediv>, <built-in function mul>, <built-in function truediv>, <built-in function sub>, <built-in function mul>)
(75, 50, 100, 2, 8, 5) (<built-in function truediv>, <built-in function mul>, <built-in function truediv>, <built-in function sub>, <built-in function truediv>)
(75, 50, 100, 2, 8, 5) (<built-in function truediv>, <built-in function mul>, <built-in function truediv>, <built-in function mul>, <built-in function add>)
(75, 50, 100, 2, 8, 5) (<built-in function truediv>, <built-in function mul>, <built-in function truediv>, <built-in function mul>, <built-in function sub>)
(75, 50, 100, 2, 8, 5) (<built-in function truediv>, <built-in function mul>, <built-in function truediv>, <built-in function mul>, <built-in function mul>)
(75, 50, 100, 2, 8, 5) (<built-in function truediv>, <

(75, 50, 100, 2, 8, 5) (<built-in function truediv>, <built-in function truediv>, <built-in function truediv>, <built-in function sub>, <built-in function truediv>)
(75, 50, 100, 2, 8, 5) (<built-in function truediv>, <built-in function truediv>, <built-in function truediv>, <built-in function mul>, <built-in function add>)
(75, 50, 100, 2, 8, 5) (<built-in function truediv>, <built-in function truediv>, <built-in function truediv>, <built-in function mul>, <built-in function sub>)
(75, 50, 100, 2, 8, 5) (<built-in function truediv>, <built-in function truediv>, <built-in function truediv>, <built-in function mul>, <built-in function mul>)
(75, 50, 100, 2, 8, 5) (<built-in function truediv>, <built-in function truediv>, <built-in function truediv>, <built-in function mul>, <built-in function truediv>)
(75, 50, 100, 2, 8, 5) (<built-in function truediv>, <built-in function truediv>, <built-in function truediv>, <built-in function truediv>, <built-in function add>)
(75, 50, 100, 2, 8, 5)

In [147]:
# Number of combinations of 5 operators with replacement.
4**5

1024

In [148]:
# No of permutations of playing numbers.
import math
math.factorial(6)

720

In [149]:
4**5 * math.factorial(6)

737280

In [150]:
# We (might not have/) haven't considered all combinations:
# RPN with (1, 2, 3, 4) and (+, -, +)...
# 1 2 3 4 + - + (-4)
# 1 2 + 3 4 - + (2)
# 1 2 3 + - 4 + (0)

#### Partitions
***

<b>Partitions</b> is a way of writing n as a sum of positive integers. Two sums that differ only in the order of their summands are considered the same partition. (If order matters, the sum becomes a composition)[[8]](https://en.wikipedia.org/wiki/Partition_(number_theory))

In [151]:
# An example list of six numbers.
numbers = [100, 75, 10, 4, 2, 1]

In [152]:
# Give all 2-partitions of a list
# where each sublist has at least one element.
def partitions(L):
  for i in range(1, len(L)):
    # Slice the list using i.
    print(L[:i], L[i:])

In [153]:
# All partitions of the numbers list.
partitions(numbers)

[100] [75, 10, 4, 2, 1]
[100, 75] [10, 4, 2, 1]
[100, 75, 10] [4, 2, 1]
[100, 75, 10, 4] [2, 1]
[100, 75, 10, 4, 2] [1]


In [154]:
# Some of the sublists in turn can be partitioned.
partitions(numbers[1:])

[75] [10, 4, 2, 1]
[75, 10] [4, 2, 1]
[75, 10, 4] [2, 1]
[75, 10, 4, 2] [1]


In [155]:
# And some of the sublists can be further (and further) parititioned>
partitions(numbers[2:])

[10] [4, 2, 1]
[10, 4] [2, 1]
[10, 4, 2] [1]


In [156]:
# We'll use generators in this circumstance.
range(100000000000)

range(0, 100000000000)

In [157]:
# Give all 2-partitions of a list
# where each sublist has  one element.
def partitions(L):
  # Check if there is no way to partition further.
  if len(L) == 1:
    yield f"{L[0]}"
  for i in range(1, len(L)):
    # Slice the list using i.
    for left, right in it.product(partitions(L[:i]), partitions(L[i:])):
      yield f"({left} ? {right})"

In [158]:
total = 0
for i in partitions(numbers):
  print(i)
  total = total + 1
print(total)

(100 ? (75 ? (10 ? (4 ? (2 ? 1)))))
(100 ? (75 ? (10 ? ((4 ? 2) ? 1))))
(100 ? (75 ? ((10 ? 4) ? (2 ? 1))))
(100 ? (75 ? ((10 ? (4 ? 2)) ? 1)))
(100 ? (75 ? (((10 ? 4) ? 2) ? 1)))
(100 ? ((75 ? 10) ? (4 ? (2 ? 1))))
(100 ? ((75 ? 10) ? ((4 ? 2) ? 1)))
(100 ? ((75 ? (10 ? 4)) ? (2 ? 1)))
(100 ? (((75 ? 10) ? 4) ? (2 ? 1)))
(100 ? ((75 ? (10 ? (4 ? 2))) ? 1))
(100 ? ((75 ? ((10 ? 4) ? 2)) ? 1))
(100 ? (((75 ? 10) ? (4 ? 2)) ? 1))
(100 ? (((75 ? (10 ? 4)) ? 2) ? 1))
(100 ? ((((75 ? 10) ? 4) ? 2) ? 1))
((100 ? 75) ? (10 ? (4 ? (2 ? 1))))
((100 ? 75) ? (10 ? ((4 ? 2) ? 1)))
((100 ? 75) ? ((10 ? 4) ? (2 ? 1)))
((100 ? 75) ? ((10 ? (4 ? 2)) ? 1))
((100 ? 75) ? (((10 ? 4) ? 2) ? 1))
((100 ? (75 ? 10)) ? (4 ? (2 ? 1)))
((100 ? (75 ? 10)) ? ((4 ? 2) ? 1))
(((100 ? 75) ? 10) ? (4 ? (2 ? 1)))
(((100 ? 75) ? 10) ? ((4 ? 2) ? 1))
((100 ? (75 ? (10 ? 4))) ? (2 ? 1))
((100 ? ((75 ? 10) ? 4)) ? (2 ? 1))
(((100 ? 75) ? (10 ? 4)) ? (2 ? 1))
(((100 ? (75 ? 10)) ? 4) ? (2 ? 1))
((((100 ? 75) ? 10) ? 4) ? (

In [159]:
# Example of ((100 ? 75) ? (10 ? (4 ? (2 ? 1)))).
((100 + 75) - (10 * (4 - (2 + 1))))

165

In [160]:
# Give all 2-partitions of a list
# where each sublist has  one element.
def patterns(numbers, operators):
  # Check if there is no way to partition further.
  if len(numbers) == 1:
    yield numbers[0]
  # Loop through all the ways to partition L into two non-empty sublists.
  for i in range(1, len(numbers)):
    # Slice the list using i.
    for left, right in it.product(patterns(numbers[:i], operators[1:i]), patterns(numbers[i:], operators[i:])):
      # Yield the next operator applied to the sublists.
      yield f'({left} {operators[0]} {right})' #[left, operators[0], right]

In [161]:
# An example list of six numbers.
numbers = [100, 75, 10, 4, 2, 1]
# Example operators.
# operators = [operator.add, operator.mul, operator.sub, operator.add, operator.add]
operators = ['+', '*', '-', '+', '+']

In [162]:
# Using eval, which mightn't be great.
for i in patterns(numbers, operators):
  print(f'{i} = {eval(i)}')

(100 + (75 * (10 - (4 + (2 + 1))))) = 325
(100 + (75 * (10 - ((4 + 2) + 1)))) = 325
(100 + (75 * ((10 + 4) - (2 + 1)))) = 925
(100 + (75 * ((10 + (4 + 2)) - 1))) = 1225
(100 + (75 * (((10 + 4) + 2) - 1))) = 1225
(100 + ((75 - 10) * (4 + (2 + 1)))) = 555
(100 + ((75 - 10) * ((4 + 2) + 1))) = 555
(100 + ((75 - (10 + 4)) * (2 + 1))) = 283
(100 + (((75 + 10) - 4) * (2 + 1))) = 343
(100 + ((75 - (10 + (4 + 2))) * 1)) = 159
(100 + ((75 - ((10 + 4) + 2)) * 1)) = 159
(100 + (((75 + 10) - (4 + 2)) * 1)) = 179
(100 + (((75 + (10 + 4)) - 2) * 1)) = 187
(100 + ((((75 + 10) + 4) - 2) * 1)) = 187
((100 * 75) + (10 - (4 + (2 + 1)))) = 7503
((100 * 75) + (10 - ((4 + 2) + 1))) = 7503
((100 * 75) + ((10 + 4) - (2 + 1))) = 7511
((100 * 75) + ((10 + (4 + 2)) - 1)) = 7515
((100 * 75) + (((10 + 4) + 2) - 1)) = 7515
((100 * (75 - 10)) + (4 + (2 + 1))) = 6507
((100 * (75 - 10)) + ((4 + 2) + 1)) = 6507
(((100 - 75) * 10) + (4 + (2 + 1))) = 257
(((100 - 75) * 10) + ((4 + 2) + 1)) = 257
((100 * (75 - (10 + 4))) 

## References
***

[[1]](https://british-game-show.fandom.com/wiki/Countdown) 
 British Game Show Wiki <br>
[[2]](https://www.cambridge.org/core/journals/journal-of-functional-programming/article/countdown-problem/FCF81E290B194D1ACBF699842D4D4CA5)  HUTTON, G. (2002). The countdown problem. Journal of Functional Programming, 12(6), 609-616. doi:10.1017/S0956796801004300 <br>
[[3]](https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.86.1168&rep=rep1&type=pdf) R. Hinze, ‘Countdown problem’. 2002. <br>
[[4]](https://arxiv.org/pdf/1502.05450.pdf) J.-M. Alliot, ‘The (Final) countdown’. arXiv, 2015.<br>
[[5]](http://doc.gold.ac.uk/aisb50/AISB50-S02/AISB50-S2-Colton-paper.pdf) Countdown Numbers Game: Solved, Analysed, Extended - Simon Colton
<br>
[[6]](https://www.mathsisfun.com/combinatorics/combinations-permutations.html)
 Combinations and Permutations
<br>
[[7]](https://mathworld.wolfram.com/ReversePolishNotation.html) Reverse Polish Notation
<br>
[[8]](https://en.wikipedia.org/wiki/Partition_(number_theory)) - Partition (number theory)
<br>
[[9]](https://www.ttested.com/polish-countdown/) A Polish Approach to Countdown

***
# End